In [ ]:
# Dataset: https://www.kaggle.com/datasets/brjapon/cwru-bearing-datasets
# CWRU: https://engineering.case.edu/bearingdatacenter/48k-drive-end-bearing-fault-data

import glob
from scipy.io import loadmat
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Show all the files that are downloaded
files = glob.glob("/content/drive/MyDrive/Colab Notebooks/CWU Optimization/cwru_archive/raw/*")
files = np.sort(files)
files

array(['/content/drive/MyDrive/Colab Notebooks/CWU Optimization/cwru_archive/raw/B007_1_123.mat',
       '/content/drive/MyDrive/Colab Notebooks/CWU Optimization/cwru_archive/raw/B014_1_190.mat',
       '/content/drive/MyDrive/Colab Notebooks/CWU Optimization/cwru_archive/raw/B021_1_227.mat',
       '/content/drive/MyDrive/Colab Notebooks/CWU Optimization/cwru_archive/raw/IR007_1_110.mat',
       '/content/drive/MyDrive/Colab Notebooks/CWU Optimization/cwru_archive/raw/IR014_1_175.mat',
       '/content/drive/MyDrive/Colab Notebooks/CWU Optimization/cwru_archive/raw/IR021_1_214.mat',
       '/content/drive/MyDrive/Colab Notebooks/CWU Optimization/cwru_archive/raw/OR007_6_1_136.mat',
       '/content/drive/MyDrive/Colab Notebooks/CWU Optimization/cwru_archive/raw/OR014_6_1_202.mat',
       '/content/drive/MyDrive/Colab Notebooks/CWU Optimization/cwru_archive/raw/OR021_6_1_239.mat',
       '/content/drive/MyDrive/Colab Notebooks/CWU Optimization/cwru_archive/raw/Time_Normal_1_098.mat'],


In [ ]:
# Load all .mat files to a dictionary typed array and check it
check_data = loadmat(files[0])
type(check_data)

dict

In [ ]:
# Show containing keys of the dataset
check_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X123_DE_time', 'X123_FE_time', 'X123RPM'])

In [ ]:
# Her dosyanın anahtarları, sürücü bitiş sinyali (DE_time), fan bitiş sinyali (FE_time) ve bazı durumlarda taban sinyali (BA) içerir. 
# X123RPM, RPM bilgisi verir. Bu çalışmada yatağa daha yakın olduğundan arızanın daha güvenilir bir göstergesi olacağı düşüncesi ile yalnızca tahrik uçları (DE) ele alınmıştır.
# Anahtarını kullanarak sürücü sonu verilerini alabiliriz.

drive_end_data = check_data["X123_DE_time"]
drive_end_data.shape

(487384, 1)

In [ ]:
# Of all the files in the folder, one file (IR014_1_175.mat) is rather peculiar. 
# It has following keys: 'X217', 'X217_DE_time', 'X175_DE_time', 'X175_FE_time', 'X175RPM'. 'X175_DE_time' 
# contains less number of data points than we need for further analysis. So we have taken 'X217_DE_time' for this particular file.
check_data_2 = loadmat(files[4])
check_data_2.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X217', 'X217_DE_time', 'X175_DE_time', 'X175_FE_time', 'X175RPM'])

In [ ]:
#Number of segments from each file by 1024
for file in files:
    data = loadmat(file)
    if file[-7:-4] == "175":         # Peculiar IR014_1 data
        key = "X217_DE_time"
    else:
        key = "X" + file[-7:-4] + "_DE_time"
    drive_end_data = data[key]
    num_segments = np.floor(len(drive_end_data)/1024)
    print(num_segments)

475.0
474.0
475.0
474.0
477.0
473.0
475.0
473.0
477.0
472.0


In [ ]:
# Neden 4600 oldugunu anlamadım
segmented_data = np.repeat(np.nan, 4600*1024).reshape(4600,1024)
num = 0
for file in files:
    data = loadmat(file)
    if file[-7:-4] == "175":    
        key = "X217_DE_time"
    else:
        key = "X" + file[-7:-4] + "_DE_time"
    drive_end_data = data[key]
    for i in range(460):           # We collect 460 segments from each file
        segmented_data[num,:] = drive_end_data[i*1024:(i+1)*1024, 0]
        num = num + 1

In [ ]:
# Check NaN values
np.sum(np.isnan(segmented_data))

0

In [ ]:
# Load Online .npz data
read_online_data = np.load("/content/drive/MyDrive/Colab Notebooks/CWU Optimization/cwru_archive/CWRU_48k_load_1_CNN_data.npz")
read_online_data.files

['data', 'labels']

In [ ]:
online_data = read_online_data["data"]
online_data.shape

(4600, 32, 32)

In [ ]:
online_data = online_data.reshape(4600, -1)
online_data.shape

(4600, 1024)

In [ ]:
np.sum(np.isclose(online_data, segmented_data, atol = 1e-4))

4710400

In [ ]:
online_data[:5,:5]

array([[-0.041097 , -0.046104 , -0.028372 ,  0.0012517,  0.033796 ],
       [ 0.048399 ,  0.048399 ,  0.050902 ,  0.050485 ,  0.040889 ],
       [ 0.18671  ,  0.31876  ,  0.39846  ,  0.41076  ,  0.36988  ],
       [-0.17378  , -0.10598  , -0.030041 ,  0.042975 ,  0.12705  ],
       [-0.0039637, -0.031501 , -0.057369 , -0.071764 , -0.064879 ]])

In [ ]:
segmented_data[:5,:5]

array([[-0.04109723, -0.046104  , -0.02837169,  0.00125169,  0.03379569],
       [ 0.04839877,  0.04839877,  0.05090215,  0.05048492,  0.04088862],
       [ 0.18671077,  0.31876431,  0.39845538,  0.41076369,  0.36987508],
       [-0.17377662, -0.10597662, -0.03004062,  0.04297477,  0.12704677],
       [-0.00396369, -0.03150092, -0.05736923, -0.07176369, -0.06487938]])

In [ ]:
segmented_data.shape

(4600, 1024)